In [5]:
import pennlinckit.data
import pennlinckit.network
import pennlinckit.utils
import pennlinckit.brain
import numpy as np
from scipy.stats import pearsonr

In [6]:
global homedir
homedir = '/cbica/home/bertolem/bpd/'

In [7]:
def score_bdp(data):
	if data.source == 'hcpd-dcan':
		reverse = [40,5,50,31,8,33]
		regular = [14,39,54,9,59,24,29,45,55,22,36,21,26,41,51,11,28,30]
		neo_name = 'nffi'

	if data.source == 'hcpya':
		reverse = ['40','05','50','31','33','08']
		regular = ['14','39','54','09','59','24','29','45','55','22','36','21','26','41','51','11','28','30']
		neo_name = 'NEORAW'

	def apply_dict(dict,x):
		return np.array(list(map(dict.get,x)))
	val_dict = {'SA':5,'A':4,'N':3,'D':2,'SD':1,'nan':np.nan}

	assert len(np.intersect1d(reverse,regular)) == 0
	assert len(np.unique(reverse)) + len(np.unique(regular)) ==24

	bpd_score = []
	for i in regular:
		x = data.subject_measures['%s_%s'%(neo_name,i)].values
		if data.source == 'hcpya':
			x = apply_dict(val_dict,x.astype(str))
		bpd_score.append(x)
	for i in reverse:
		x = data.subject_measures['%s_%s'%(neo_name,i)].values
		if data.source == 'hcpya':
			x = apply_dict(val_dict,x.astype(str))
		x = x * -1
		bpd_score.append(x)
	bpd_score = np.nanmean(bpd_score,axis=0)
	data.subject_measures['bpd_score'] = bpd_score

In [8]:
def make_data(source,cores=10):
	"""
	Make the datasets and run network metrics
	"""
	data = pennlinckit.data.dataset(source,task='**', parcels='Schaefer417',fd_scrub=.2)
	data.load_matrices()
	data.filter(way='>',value=100,column='n_frames')
	data.network = pennlinckit.network.make_networks(data,yeo_partition=7,cores=cores-1)
	pennlinckit.utils.save_dataset(data,'/{0}/data/{1}.data'.format(homedir,source))

In [11]:
source = 'hcpya'
make_data(source)
data = pennlinckit.utils.load_dataset('/{0}/data/{1}.data'.format(homedir,source))
score_bdp(data)
data.filter(way='has_subject_measure',value='bpd_score')
regressors = ['meanFD','interview_age','sex']

In [ ]:
data.targets = data.subject_measures['bpd_score'].values
accuracy = np.zeros(data.matrix.shape[1])
bootstrap = np.zeros(data.matrix.shape[1])
for node in range(data.matrix.shape[1]):
	data.features = data.matrix[:,node]
	pennlinckit.utils.predict(data,**{'model':'ridge','cv':'KFold','folds':10,'remove_linear_vars':regressors})
	# pennlinckit.utils.predict(data,**{'model':'deep','cv':'KFold','folds':10,'neurons':400,'layers':10,'remove_linear_vars':regressors})
	r,low,high,p = pennlinckit.utils.bootstrap_corr(data.targets,data.prediction,pearsonr,1000)
	accuracy[node] = r 
	bootstrap[node] = p

In [ ]:
colors = np.array(pennlinckit.utils.make_heatmap(pennlinckit.utils.cut_data(accuracy,1.5),sns.diverging_palette(220, 10,n=1001)))
out_path='/{0}/brains/{1}_{2}_prediction_acc'.format(homedir,source,'_'.join(regressors))
pennlinckit.brain.write_cifti(colors,out_path,parcels='Schaefer400')